In [12]:
# Import libraries
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
import csv
warnings.filterwarnings("ignore")

In [13]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [14]:
# Create dataframe
cities_df = pd.read_csv(cities_path, usecols = ["city"])

In [15]:
cities_df.drop_duplicates(inplace = True)

In [16]:
city_list = cities_df["city"].tolist()

In [17]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [53]:
# List lenght
city_list[-1]

'Kolsassberg, Austria'

In [58]:
city_list[:2399]

'Novaci, Romania'

In [59]:
city_list = city_list[:2399]
len(city_list)

2399

In [62]:
# This process does 10000 to 20000
len(city_list)

0

In [61]:
options = Options()
options.headless = True
options.add_argument("--window-size=1024x768")
driver = webdriver.Chrome('/usr/bin/chromedriver', chrome_options=options)
url = "https://www.hotels.com/"

total_elapsed_time = 0
cities_processed = 0

with open(r'../data/hotels_com_scrape_v3_silvia.csv', 'a') as f:
    csv_writer = csv.writer(f)
    for attempt in range(100):
        try:
            for city in list(city_list):
                start_time = time.time()
                print(city, end = " - ")

                searchKey = city
                checkInDate = '08/27/2020'
                checkOutDate = '08/29/2020'

                driver.get(url)

                searchKeyElement = driver.find_elements_by_name("q-destination")
                checkInElement = driver.find_elements_by_name("q-localised-check-in")
                checkOutElement = driver.find_elements_by_name("q-localised-check-out")
                submitButton = driver.find_elements_by_css_selector("button.cta-strong")

                if searchKeyElement and checkInElement and checkOutElement:
                    checkInElement[0].clear()
                    checkInElement[0].send_keys(checkInDate)
                    checkOutElement[0].clear()
                    checkOutElement[0].send_keys(checkOutDate)
                    searchKeyElement[0].clear()
                    searchKeyElement[0].send_keys(searchKey)
                    time.sleep(2)

                    try:
                        submitButton[0].click()
                    except:                
                        # Try to close the the autocomplete box
                        searchKeyElement[0].send_keys(Keys.ARROW_DOWN)
                        searchKeyElement[0].send_keys(Keys.ENTER)
                        submitButton[0].click()

                    time.sleep(2)

                    page_length = 0
                    last_page_length = 0
                    for i in range(5):
                        page_length = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var pageLength=document.body.scrollHeight;return pageLength;")
                        time.sleep(1)
                        if last_page_length == page_length:
                            break
                        last_page_length = page_length

                    hotels = driver.find_elements_by_css_selector('section.hotel-wrap')

                    for index, hotel in enumerate(hotels):
                        hotel_name = hotel.find_element_by_css_selector('a.property-name-link').text.strip()
                        hotel_url = hotel.find_element_by_css_selector('a.property-name-link').get_attribute("href")
                        try:
                            hotel_rating = float(hotel.find_element_by_css_selector('div.ta-logo').text.split()[-1])
                            hotel_reviews = int(hotel.find_element_by_css_selector('div.ta-total-reviews').text.split()[0])
                        except:
                            hotel_rating = "na"
                            hotel_reviews = "na"
                        hotel_address = hotel.find_element_by_css_selector('span.address').text
                        hotel_locality = hotel.find_element_by_css_selector("a.map-link").text.strip()
                        hotel_landmarks = hotel.find_element_by_css_selector("ul.property-landmarks").text.replace("\n", ", ")
                        try:
                            hotel_price = float(hotel.find_element_by_css_selector("a.price-link").find_element_by_tag_name("ins").text.strip("$"))
                        except:
                            hotel_price = "na"

                        row = [city, hotel_name, hotel_rating, hotel_reviews, hotel_address, hotel_locality, hotel_landmarks, hotel_price, hotel_url]
                        csv_writer.writerow(row)
                    print(str(len(hotels)) + " hotels found.", end = " ")
                city_list.remove(city)
                cities_processed = cities_processed + 1
                end_time = time.time()
                elapsed_time = end_time - start_time
                total_elapsed_time = total_elapsed_time + elapsed_time
                average_elapsed_time = total_elapsed_time / cities_processed
                time_left = average_elapsed_time * len(city_list)
                print(time.strftime("Time left: %d days %H hours %M minutes", time.gmtime(time_left)))
        except Exception as e:
            print(e)
            print("Stuff broke, restarting and retrying.")
            continue
        break

driver.quit()
print("We done.")

Sabang City, Indonesia - 6 hotels found. Time left: 01 days 07 hours 15 minutes
Wonosobo, Indonesia - 1 hotels found. Time left: 01 days 07 hours 09 minutes
Bukit Raja, Malaysia - 0 hotels found. Time left: 01 days 06 hours 28 minutes
Strzyzów, Poland - 8 hotels found. Time left: 01 days 06 hours 52 minutes
Flandes, Colombia - 3 hotels found. Time left: 01 days 07 hours 00 minutes
Jinsha Township, Taiwan - 11 hotels found. Time left: 01 days 07 hours 03 minutes
North Hollywood, USA - 61 hotels found. Time left: 01 days 07 hours 44 minutes
Bukit Tinggi, Malaysia - 52 hotels found. Time left: 01 days 08 hours 16 minutes
Big Sur, USA - 1 hotels found. Time left: 01 days 08 hours 00 minutes
Le Cannet, France - 52 hotels found. Time left: 01 days 08 hours 28 minutes
Kirkcaldy, United Kingdom - 32 hotels found. Time left: 01 days 08 hours 43 minutes
Praia da Luz, Portugal - 6 hotels found. Time left: 01 days 08 hours 30 minutes
Barahona, Dominican Republic - 1 hotels found. Time left: 01 day

Cadrezzate, Italy - 52 hotels found. Time left: 01 days 08 hours 08 minutes
Prince Rupert, Canada - 1 hotels found. Time left: 01 days 08 hours 07 minutes
Bad Gandersheim, Germany - 32 hotels found. Time left: 01 days 08 hours 08 minutes
Tiers am Rosengarten, Italy - 0 hotels found. Time left: 01 days 08 hours 06 minutes
Geiselwind, Germany - 18 hotels found. Time left: 01 days 08 hours 07 minutes
Trois-Ponts, Belgium - 62 hotels found. Time left: 01 days 08 hours 09 minutes
Ahrensfelde-Blumberg, Germany - 7 hotels found. Time left: 01 days 08 hours 08 minutes
Greven, Germany - 12 hotels found. Time left: 01 days 08 hours 07 minutes
Bergkamen, Germany - 53 hotels found. Time left: 01 days 08 hours 09 minutes
Trenton, Canada - 11 hotels found. Time left: 01 days 08 hours 09 minutes
Circleville, USA - 1 hotels found. Time left: 01 days 08 hours 08 minutes
Hittisau, Austria - 34 hotels found. Time left: 01 days 08 hours 09 minutes
Grieskirchen, Austria - 0 hotels found. Time left: 01 days

Pagudpud, Philippines - 2 hotels found. Time left: 01 days 07 hours 53 minutes
Manhattan Beach, USA - 62 hotels found. Time left: 01 days 07 hours 54 minutes
Ceyreste, France - 51 hotels found. Time left: 01 days 07 hours 55 minutes
Beerse, Belgium - 33 hotels found. Time left: 01 days 07 hours 55 minutes
Mittersill, Austria - 32 hotels found. Time left: 01 days 07 hours 55 minutes
Karviná, Czech Republic - 17 hotels found. Time left: 01 days 07 hours 55 minutes
Epen, Netherlands - 41 hotels found. Time left: 01 days 07 hours 55 minutes
Hyde Park, USA - 44 hotels found. Time left: 01 days 07 hours 56 minutes
Ruma, Serbia - 6 hotels found. Time left: 01 days 07 hours 55 minutes
Manisa, Turkey - 3 hotels found. Time left: 01 days 07 hours 54 minutes
Cradle Mountain, Australia - 1 hotels found. Time left: 01 days 07 hours 54 minutes
Thiviers, France - 2 hotels found. Time left: 01 days 07 hours 53 minutes
Ras Al Khafji, Saudi Arabia - 1 hotels found. Time left: 01 days 07 hours 53 minutes

Whitehouse, Jamaica - 17 hotels found. Time left: 01 days 07 hours 30 minutes
Vila Velha de Ródão, Portugal - 6 hotels found. Time left: 01 days 07 hours 30 minutes
Streetsboro, USA - 7 hotels found. Time left: 01 days 07 hours 29 minutes
Colombiers, France - 56 hotels found. Time left: 01 days 07 hours 30 minutes
Vetlanda, Sweden - 4 hotels found. Time left: 01 days 07 hours 29 minutes
Bricquebec, France - 25 hotels found. Time left: 01 days 07 hours 29 minutes
Chilliwack, Canada - 3 hotels found. Time left: 01 days 07 hours 28 minutes
Ried im Innkreis, Austria - 14 hotels found. Time left: 01 days 07 hours 28 minutes
Bhadrak, India - 0 hotels found. Time left: 01 days 07 hours 27 minutes
Teton Village, USA - 15 hotels found. Time left: 01 days 07 hours 27 minutes
Lerwick, United Kingdom - 5 hotels found. Time left: 01 days 07 hours 27 minutes
Kristiansund, Norway - 7 hotels found. Time left: 01 days 07 hours 26 minutes
Killaloe, Ireland - 13 hotels found. Time left: 01 days 07 hours 

Vällingby, Sweden - 6 hotels found. Time left: 01 days 07 hours 00 minutes
Jinotega, Nicaragua - 1 hotels found. Time left: 01 days 07 hours 00 minutes
Winford, United Kingdom - 62 hotels found. Time left: 01 days 07 hours 00 minutes
Wirksworth, United Kingdom - 5 hotels found. Time left: 01 days 06 hours 59 minutes
La Salle-les-Alpes, France - 3 hotels found. Time left: 01 days 06 hours 59 minutes
Bueno Brandão, Brazil - 3 hotels found. Time left: 01 days 06 hours 58 minutes
Bar-sur-Seine, France - 21 hotels found. Time left: 01 days 06 hours 58 minutes
Namhae, South Korea - 2 hotels found. Time left: 01 days 06 hours 58 minutes
Makawao, USA - 17 hotels found. Time left: 01 days 06 hours 58 minutes
Lake Geneva, USA - 4 hotels found. Time left: 01 days 06 hours 57 minutes
Lichtentanne, Germany - 25 hotels found. Time left: 01 days 06 hours 57 minutes
Holzwickede, Germany - 51 hotels found. Time left: 01 days 06 hours 57 minutes
Vimercate, Italy - 32 hotels found. Time left: 01 days 06 

...

Bishopton, United Kingdom - 62 hotels found. Time left: 01 days 01 hours 34 minutes
Pineda de Mar, Spain - 49 hotels found. Time left: 01 days 01 hours 34 minutes
Argenbühl, Germany - 42 hotels found. Time left: 01 days 01 hours 34 minutes
Viloví de Oñar, Spain - 52 hotels found. Time left: 01 days 01 hours 33 minutes
Sevran, France - 32 hotels found. Time left: 01 days 01 hours 33 minutes
Iznájar, Spain - 3 hotels found. Time left: 01 days 01 hours 33 minutes
Arrochar, United Kingdom - 5 hotels found. Time left: 01 days 01 hours 33 minutes
Geseke, Germany - 34 hotels found. Time left: 01 days 01 hours 33 minutes
Eagle, USA - 7 hotels found. Time left: 01 days 01 hours 32 minutes
Reil, Germany - 55 hotels found. Time left: 01 days 01 hours 32 minutes
St. Jakob im Defereggental, Austria - 6 hotels found. Time left: 01 days 01 hours 32 minutes
Gries im Sellrain-Praxmar, Austria - 45 hotels found. Time left: 01 days 01 hours 32 minutes
Hopferau, Germany - 5 hotels found. Time left: 01 day

Beringen, Belgium - 32 hotels found. Time left: 01 days 01 hours 12 minutes
Biskupiec , Poland - 4 hotels found. Time left: 01 days 01 hours 12 minutes
Bull Bay, Jamaica - 34 hotels found. Time left: 01 days 01 hours 11 minutes
Greensburg, USA - 3 hotels found. Time left: 01 days 01 hours 11 minutes
Aversa, Italy - 42 hotels found. Time left: 01 days 01 hours 11 minutes
Krumovgrad, Bulgaria - 0 hotels found. Time left: 01 days 01 hours 11 minutes
Gattinara, Italy - 52 hotels found. Time left: 01 days 01 hours 11 minutes
Słupca, Poland - 6 hotels found. Time left: 01 days 01 hours 10 minutes
Pasym , Poland - 1 hotels found. Time left: 01 days 01 hours 10 minutes
Belzig, Germany - 14 hotels found. Time left: 01 days 01 hours 10 minutes
Solna, Sweden - 6 hotels found. Time left: 01 days 01 hours 10 minutes
Yancheng District, Taiwan - 3 hotels found. Time left: 01 days 01 hours 10 minutes
Shiroishi, Japan - 1 hotels found. Time left: 01 days 01 hours 09 minutes
Omuta, Japan - 1 hotels foun

Houli District, Taiwan - 6 hotels found. Time left: 01 days 00 hours 50 minutes
Sainte-Agathe-des-Monts, Canada - 58 hotels found. Time left: 01 days 00 hours 50 minutes
Juno Beach, USA - 5 hotels found. Time left: 01 days 00 hours 49 minutes
Kerpen, Germany - 42 hotels found. Time left: 01 days 00 hours 49 minutes
Suvorovo, Bulgaria - 6 hotels found. Time left: 01 days 00 hours 49 minutes
Dalpe, Switzerland - 22 hotels found. Time left: 01 days 00 hours 49 minutes
Ciudad Victoria, Mexico - 4 hotels found. Time left: 01 days 00 hours 49 minutes
Ocean Isle Beach, USA - 8 hotels found. Time left: 01 days 00 hours 48 minutes
Marfa, USA - 8 hotels found. Time left: 01 days 00 hours 48 minutes
Winterswijk, Netherlands - 28 hotels found. Time left: 01 days 00 hours 48 minutes
Turégano, Spain - 23 hotels found. Time left: 01 days 00 hours 48 minutes
Romorantin-Lanthenay, France - 50 hotels found. Time left: 01 days 00 hours 47 minutes
Alahärmä, Finland - 4 hotels found. Time left: 01 days 00 

Hebron, USA - 1 hotels found. Time left: 01 days 00 hours 31 minutes
Radcliff, USA - 1 hotels found. Time left: 01 days 00 hours 30 minutes
Bollène, France - 62 hotels found. Time left: 01 days 00 hours 30 minutes
Bellerive-sur-Allier, France - 12 hotels found. Time left: 01 days 00 hours 30 minutes
Parsberg, Germany - 25 hotels found. Time left: 01 days 00 hours 30 minutes
Rheda-Wiedenbrück, Germany - 22 hotels found. Time left: 01 days 00 hours 29 minutes
Castries, France - 62 hotels found. Time left: 01 days 00 hours 29 minutes
Praia de Esmoriz, Portugal - 25 hotels found. Time left: 01 days 00 hours 29 minutes
Pruszków, Poland - 51 hotels found. Time left: 01 days 00 hours 29 minutes
Capbreton, France - 42 hotels found. Time left: 01 days 00 hours 29 minutes
Albano Laziale, Italy - 6 hotels found. Time left: 01 days 00 hours 28 minutes
Ceská Kamenice, Czech Republic - 19 hotels found. Time left: 01 days 00 hours 28 minutes
Gudvangen, Norway - 2 hotels found. Time left: 01 days 00 h

Sassenheim, Netherlands - 21 hotels found. Time left: 01 days 00 hours 10 minutes
Indaiatuba, Brazil - 4 hotels found. Time left: 01 days 00 hours 10 minutes
Jausiers, France - 1 hotels found. Time left: 01 days 00 hours 09 minutes
Racławice, Poland - 22 hotels found. Time left: 01 days 00 hours 09 minutes
Dachstein, France - 52 hotels found. Time left: 01 days 00 hours 09 minutes
Ruzayevka, Russia - 1 hotels found. Time left: 01 days 00 hours 09 minutes
Giby, Poland - 1 hotels found. Time left: 01 days 00 hours 09 minutes
Pöttmes, Germany - 20 hotels found. Time left: 01 days 00 hours 08 minutes
Guelmim, Morocco - 0 hotels found. Time left: 01 days 00 hours 08 minutes
Le Blanc, France - 42 hotels found. Time left: 01 days 00 hours 08 minutes
Millerovo, Russia - 0 hotels found. Time left: 01 days 00 hours 08 minutes
Alphen aan den Rijn, Netherlands - 42 hotels found. Time left: 01 days 00 hours 08 minutes
Lengenwang, Germany - 31 hotels found. Time left: 01 days 00 hours 07 minutes
Hig

In [15]:
driver.quit()